In [16]:
import torch 
import torch.nn as nn
from typing import List, Union, Tuple, Dict, Optional

import numpy as np

from models import Fusion

from transformers import AutoTokenizer
from transformers import BertModel

from fusion import Fusion

In [2]:
# vision_pretrained = "google/vit-base-patch16-224"
text_pretrained = "bert-base-uncased"

vision_model = xrv.autoencoders.ResNetAE(weights="101-elastic")
text_model = BertModel.from_pretrained(text_pretrained)

model = Fusion(
    text_model=text_model,
    vision_model=vision_model,
    vision_model_type='ae', 
    vision_output_dim=512,
    projection_dim=768,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
tokenizer = AutoTokenizer.from_pretrained(text_pretrained)
image_processor = ae_image_processor

img_path = 'sample.jpeg'
img = skimage.io.imread(img_path)
imgs = np.stack([img, img])
vision_inputs = image_processor(imgs)

text_inputs = tokenizer(
    text=["Nodule", "Lung Lesion"], 
    return_tensors="pt", padding=True, 
)
inputs = {**vision_inputs, **text_inputs}
outputs = model(**inputs, return_dict=True, return_loss=True)